In [99]:
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor

import numpy as np

# 定义文件路径
file_path = 'vdjdb.txt'  # 将 'your_file.txt' 替换为你的文件路径

# 读取文件内容
with open(file_path, 'r', encoding='utf-8') as file:
    # 读取文件的第一行，获取所有的信息变量名
    header = file.readline().strip().split('\t')
    tcr_data = [dict(zip(header, line.strip().split('\t'))) for line in file]
print(header)
cdr3_dict = {}
for row in tcr_data:
    complex_id = row['complex.id']
    cdr3 = row['cdr3']
    # 将相同 complex.id 的 cdr3 拼接起来
    if complex_id in cdr3_dict:
        cdr3_dict[complex_id].append(cdr3)
    else:
        cdr3_dict[complex_id] = [cdr3]
# 假设有一个包含 TCR 序列的 DataFrame
for row in tcr_data:
    complex_id = row['complex.id']
    antigen_epitope = row['antigen.epitope']
    vdjdb_score = row['vdjdb.score']
    species=row['species']
    # 将相同 complex.id 的 cdr3 拼接起来
    if len(cdr3_dict[complex_id]) == 2:
        cdr3_dict[complex_id].append(antigen_epitope)
        cdr3_dict[complex_id].append(vdjdb_score)
        cdr3_dict[complex_id].append(species)
    else:
        continue
cdr3_dict.pop('0')
##删除未配对的TCR
df_cdr3 = pd.DataFrame(cdr3_dict)
df_cdr3_trans = df_cdr3.transpose()
names = ['TRA', 'TRB', 'antigen_epitope', 'vdjdb.score','species']
df_cdr3_trans.columns = names
print(df_cdr3_trans)

['complex.id', 'gene', 'cdr3', 'v.segm', 'j.segm', 'species', 'mhc.a', 'mhc.b', 'mhc.class', 'antigen.epitope', 'antigen.gene', 'antigen.species', 'reference.id', 'method', 'meta', 'cdr3fix', 'vdjdb.score', 'web.method', 'web.method.seq', 'web.cdr3fix.nc', 'web.cdr3fix.unmp']
                  TRA                   TRB antigen_epitope vdjdb.score  \
1       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEKGGL           2   
2      CAVPSGAGSYQLTF   CASSFEPGQGFYSNQPQHF        FLKEKGGL           2   
3         CAVKASGSRLT  CASSYEPGQVSHYSNQPQHF        FLKEKGGL           2   
4       CAYRPPGTYKYIF        CASSALASLNEQFF        FLKEKGGL           2   
5       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEQGGL           2   
...               ...                   ...             ...         ...   
30590   CMDEGGSNYKLTF         CASSVRSTDTQYF    PQPELPYPQPQL           0   
30591     CSLYNNNDMRF         CASSLRYTDTQYF    PQPELPYPQPQL           0   
30592   CALSTDSWGKLQF       CASSPGQGGDNEQFF   PQ

In [100]:
df_cdr3_trans=df_cdr3_trans[df_cdr3_trans['species'] == 'HomoSapiens']
neg_data= df_cdr3_trans[df_cdr3_trans['vdjdb.score'] == '0']
pos_data_1= df_cdr3_trans[df_cdr3_trans['vdjdb.score'] == '1']
pos_data_2= df_cdr3_trans[df_cdr3_trans['vdjdb.score'] == '2']
pos_data_3= df_cdr3_trans[df_cdr3_trans['vdjdb.score'] == '3']
# 复制 pos_data_2 的数据两次
pos_data_2_copy = pos_data_2.copy()
pos_data_2_copy = pd.concat([pos_data_2_copy] * 2, ignore_index=True)

# 复制 pos_data_3 的数据三次
pos_data_3_copy = pos_data_3.copy()
pos_data_3_copy = pd.concat([pos_data_3_copy] * 3, ignore_index=True)

# 将复制后的数据与 pos_data_1 拼接在一起
pos_data = pd.concat([pos_data_1, pos_data_2_copy, pos_data_3_copy], ignore_index=True)
# 确定阳性样本数量
num_positive_samples = len(pos_data)

# 从阴性样本中随机抽取与阳性样本数量相同的样本
neg_data_sampled = neg_data.sample(n=num_positive_samples, random_state=42)
neg_data_sampled = neg_data_sampled.reset_index(drop=True)
pos_data = pos_data.reset_index(drop=True)
neg_data_sampled['label']=0
pos_data['label']=1

In [101]:
balanced_dataset = pd.concat([neg_data_sampled, pos_data], axis=0)
balanced_dataset = balanced_dataset.reset_index(drop=True)
balanced_dataset['TRA_TRB_Combined'] = balanced_dataset["TRA"] + balanced_dataset["TRB"]

In [102]:
##----------------接下来编码-------------------------------
encoding_map = {'A': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'C': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'D': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'E': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'F': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'G': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'H': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'I': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'K': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'L': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'M': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                'N': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                'P': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                'Q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                'R': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                'S': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                'T': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                'V': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                'W': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                'Y': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}
cdr3_encoded = [[encoding_map[char] for char in sequence] for sequence in balanced_dataset['TRA_TRB_Combined']]
antigen_encoded = [[encoding_map[char] for char in sequence] for sequence in balanced_dataset['antigen_epitope']]
##独热码成功编辑，但是矩阵长度不一致
longest_cdr3 = max(balanced_dataset['TRA_TRB_Combined'], key=len)
print("最长的cdr3:", longest_cdr3)
print("最长cdr3的长度:", len(longest_cdr3))
longest_antigen_epitope = max(balanced_dataset['antigen_epitope'], key=len)
print("最长的antigen_epitope:", longest_antigen_epitope)
print("最长antigen_epitope的长度:", len(longest_antigen_epitope))


def padding_sequence(origin, sequence_length):
    padded = np.zeros((sequence_length, 20))
    padded[:len(origin)] = origin
    return padded


cdr3_encoded_padded = [padding_sequence(seq, len(longest_cdr3)) for seq in cdr3_encoded]
antigen_encoded_padded = [padding_sequence(seq, len(longest_antigen_epitope)) for seq in antigen_encoded]
cdr3_encoded_padded_flat = [seq.flatten() for seq in cdr3_encoded_padded]
cdr3_length=[len(seq) for seq in balanced_dataset['TRA_TRB_Combined']]
cdr3_encoded_padded_flat_with_length = [np.concatenate((flat_seq, [seq_length])) for flat_seq, seq_length in zip(cdr3_encoded_padded_flat, cdr3_length)]
antigen_encoded_padded_flat = [seq.flatten() for seq in antigen_encoded_padded]
antigen_length=[len(seq) for seq in balanced_dataset['antigen_epitope']]
antigen_encoded_padded_flat_with_length = [np.concatenate((flat_seq, [seq_length])) for flat_seq, seq_length in zip(antigen_encoded_padded_flat, antigen_length)]
balanced_dataset['cdr3_code'] = cdr3_encoded_padded_flat_with_length
balanced_dataset['antigen_code'] =antigen_encoded_padded_flat_with_length

最长的cdr3: CAGAFWQGAQKLVFCASSQAGQRLAGALNPTQSYNEQFF
最长cdr3的长度: 39
最长的antigen_epitope: MTEYKLVVVGAVGVGKSALTIQLI
最长antigen_epitope的长度: 24


In [103]:
balanced_dataset['input'] = balanced_dataset.apply(lambda row: list(row['cdr3_code']) + list(row['antigen_code']), axis=1)

In [104]:
# 划分特征和目标变量
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X = np.array(balanced_dataset['input'].tolist())  # cdr3和待选抗原作为特征
y = np.array(balanced_dataset['label'].tolist())  # label做为标签
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=516)

In [106]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
# 初始化决策树模型
decision_tree = DecisionTreeClassifier(random_state=22)

# 在训练集上拟合模型
decision_tree.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = decision_tree.predict(X_test)

# 计算准确率
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Recall: 0.9293966623876765
Precision: 0.8960396039603961
F1 Score: 0.9124133585381222
Accuracy: 0.9133416458852868


In [107]:
##Logistic Regression Model
# 使用逻辑回归模型进行分类
from sklearn.linear_model import LogisticRegression

# 初始化逻辑回归模型
logistic_regression = LogisticRegression(max_iter=1000,random_state=22)

# 训练模型
logistic_regression.fit(X_train, y_train)

# 预测测试集
y_pred = logistic_regression.predict(X_test)

# 评估模型性能
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Recall: 0.9242618741976893
Precision: 0.8674698795180723
F1 Score: 0.8949658172778123
Accuracy: 0.8946384039900249


In [115]:
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassfier


random_forest_model = RandomForestClassifier(n_estimators=200,max_depth=None, max_features='log2', min_samples_split=2,random_state=22)
random_forest_model.fit(X_train, y_train)

# predict test
y_pred = random_forest_model.predict(X_test)
# accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Recall: 0.9563543003851092
Precision: 0.905224787363305
F1 Score: 0.9300873907615481
Accuracy: 0.9301745635910225


In [116]:
##Naive_bayes Model
from sklearn.naive_bayes import GaussianNB
# 初始化朴素贝叶斯模型
naive_bayes = GaussianNB()

# 在训练集上拟合模型
naive_bayes.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = naive_bayes.predict(X_test)

# 计算准确率
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Recall: 0.2785622593068036
Precision: 0.9273504273504274
F1 Score: 0.4284304047384009
Accuracy: 0.6390274314214464


In [117]:
#KNN Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 初始化KNN模型
knn_model = KNeighborsClassifier(n_neighbors=1)

# 训练模型
knn_model.fit(X_train, y_train)

# 预测
y_pred = knn_model.predict(X_test)

# 计算准确率
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Recall: 0.938382541720154
Precision: 0.9103362391033624
F1 Score: 0.9241466498103665
Accuracy: 0.9251870324189526
